# i. Introduction

Notebook ini merupakan salah satu program yang dibuat untuk melakukan data scraping salah satu brand TWS (True Wireless Stereo) pada E-Commerce Amazon, API-key didapatkan dari ScraperAPI, Looping page dibatasi sebanyak 5 kali

# ii. Import Libraies

In [ ]:
# Import Libraries
import pandas as pd
import json
import time
from bs4 import BeautifulSoup
import requests

# iii. Data Scraping

In [ ]:
# Scraping dengan ScraperAPI
api_key = "e3b7e2ea13b802e62478348959b1e3c2"
response=[]

# Looping untuk 5 halaman
for page in range(1,6):
    url = "https://www.amazon.com/s?k=soundcore+tws&page={page}&xpid=UU5wP5pA5IhP5&crid=2DWSTTNK4TZEU&qid=1767542732&sprefix=soundcore+tw%2Caps%2C340&ref=sr_pg_{page}"

    # Token API
    payload = {
    'api_key': api_key,
    'url': url,
    'country_code': 'us',  
    'render': 'false'      
    }

    # Error Handling
    try:
        res = requests.get("https://api.scraperapi.com", params=payload, timeout=15)
        res.raise_for_status()  
        response.append(res)
        print(f"Sukses mengambil halaman {page}")
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil halaman {page}: {e}")
        continue  
    
    # Time delay
    time.sleep(5)  

Sukses mengambil halaman 1
Sukses mengambil halaman 2
Sukses mengambil halaman 3
Sukses mengambil halaman 4
Sukses mengambil halaman 5


In [ ]:
# Extracting Product Links
product_url = []

for res in response:
    """
    Perulangan untuk membuka data hasil scraping
    """
    extractor = BeautifulSoup(res.text, 'html.parser')

    # Get hyperlink product
    links = extractor.find_all('a', href=True)
    for link in links:
        href = link['href']

        if (
            "/dp/" in href and
            not href.startswith("/gp/") and
            "aax-us-iad" not in href and
            "slredirect" not in href
        ):
            # Membuat link URL
            base_url = "https://www.amazon.com"
            product_link = href.split('?')[0]  
            full_url = base_url + product_link

            # Hindari duplikat
            if full_url not in product_url:
                product_url.append(full_url)

# Menampilkan hasil link produk
for i, url in enumerate(product_url, start=1):
    print(f"{i}. {url}")

1. https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTYZG5C/ref=sr_1_1
2. https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTYZG5C/ref=cs_sr_dp_1
3. https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTPF7CZ/ref=cs_sr_dp_2
4. https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTR3PMF/ref=cs_sr_dp_3
5. https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTF579W/ref=cs_sr_dp_4
6. https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTN7C8C/ref=cs_sr_dp_5
7. https://www.amazon.com/Soundcore-Wireless-Bluetooth-Water-Resistant-Customization/dp/B0BTYCRJSS/ref=sr_1_3
8. https://www.amazon.com/Soundcore-Wireless-Bluetooth-Water-Resistant-Customization/dp/B0BTYCRJSS/ref=cs_sr_dp_1
9. https://www.amazon.com/Soundcore-Wireless-Bluetooth-Water-Resistant-Customization/dp/B0BTYDLTM3/ref=cs_sr_dp_2
10. https://www.amazon.com/Soundcore-Wirel

In [ ]:
# Scraping Review Produk
scraping_review = []

# Looping untuk setiap link produk
for page in product_url:
    
    # Token API
    payload = {
    'api_key': api_key,
    'url': page,
    'country_code': 'us',  
    'render': 'true'      
    }

    # Error Handling
    try:
        response = requests.get("https://api.scraperapi.com", params=payload, timeout=15)
        response.raise_for_status()  
        scraping_review.append(response)
        print(f"Sukses: {page}")
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil {page}: {e}")
        continue  
    
    # Time Delay
    time.sleep(5)  


Sukses: https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTYZG5C/ref=sr_1_1
Sukses: https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTYZG5C/ref=cs_sr_dp_1
Sukses: https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTPF7CZ/ref=cs_sr_dp_2
Sukses: https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTR3PMF/ref=cs_sr_dp_3
Sukses: https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTF579W/ref=cs_sr_dp_4
Sukses: https://www.amazon.com/Soundcore-Cancelling-Powerful-Playtime-Bluetooth/dp/B0CRTN7C8C/ref=cs_sr_dp_5
Sukses: https://www.amazon.com/Soundcore-Wireless-Bluetooth-Water-Resistant-Customization/dp/B0BTYCRJSS/ref=sr_1_3
Sukses: https://www.amazon.com/Soundcore-Wireless-Bluetooth-Water-Resistant-Customization/dp/B0BTYCRJSS/ref=cs_sr_dp_1
Sukses: https://www.amazon.com/Soundcore-Wireless-Bluetooth-Water-Resistant-Customization/dp/B0BTYDLTM3/ref=cs_sr_dp

In [ ]:
# List for review data
review_data =[]

for product in scraping_review:

    # BS4 Parser
    parser = BeautifulSoup(product.text, 'html.parser')

    # Parsing Procut Title
    title = parser.find(id='productTitle')
    product_title=title.get_text(strip=True) if title else 'N/A'

    # Parsing product Image URL
    img_tag = parser.select_one('#imgTagWrapperId img')
    image_url = img_tag['src'] if img_tag else 'N/A'

    # Parsing Price
    price = parser.select_one('.a-price .a-offscreen')
    product_price = price.get_text(strip=True) if price else 'N/A'

    # Parsing Rating
    rating = parser.find('span', {'class': 'a-icon-alt'})
    product_rating=rating.get_text(strip=True) if rating else 'N/A'

    # Parsing Brand 
    brand_row = parser.find("tr", {"class": "a-spacing-small po-brand"})
    brand_td = brand_row.find("td", class_="a-span9") if brand_row else None
    brand = brand_td.get_text(strip=True) if brand_td else 'N/A'

    # Parsing Color 
    color_row = parser.find("tr", {"class": "a-spacing-small po-color"})
    color_td = color_row.find("td", class_="a-span9") if color_row else None
    color = color_td.get_text(strip=True) if color_td else 'N/A'

    # Parsing Ear Placement
    placement_row = parser.find("tr", {"class": "a-spacing-small po-headphones_ear_placement"})
    placement_td = placement_row.find("td", class_="a-span9") if placement_row else None
    placement = placement_td.get_text(strip=True) if placement_td else 'N/A'

    # Parsing Form Factor
    form_factor_row = parser.find("tr", {"class": "a-spacing-small po-headphones_form_factor"})
    form_factor_td = form_factor_row.find("td", class_="a-span9") if form_factor_row else None
    form_factor = form_factor_td.get_text(strip=True) if form_factor_td else 'N/A'

    # Parsing Impedance
    impedance_row = parser.find("tr", {"class": "a-spacing-small po-impedance"})
    impedance_td = impedance_row.find("td", class_="a-span9") if impedance_row else None
    impedance = impedance_td.get_text(strip=True) if impedance_td else 'N/A'

    # Get Data Review
    reviews = []
    review_blocks = parser.find_all('li', {'data-hook': 'review'})
    for block in review_blocks:

       # Rating review
        rating_val = "N/A"
        rating_tag = block.find('i', {'data-hook': 'review-star-rating'})
        if rating_tag:
            span = rating_tag.find('span', class_='a-icon-alt')
            if span:
                rating_val = span.get_text(strip=True)

        # Review text
        review_text = "N/A"
        review_body = block.find('span', {'data-hook': 'review-body'})
        if review_body:
            collapsed = review_body.find('div', {'data-hook': 'review-collapsed'})
            if collapsed:
                text_span = collapsed.find('span')
                if text_span:
                    review_text = text_span.get_text(strip=True)

        if review_text != "N/A" and rating_val != "N/A":
            reviews.append({
                "review": review_text,
                "rating": rating_val
            })

    # Skip if no reviews found
    if not reviews:
        continue  

    # Make Dictionary for review data
    dict_review = {
        "product_name" : product_title,
        "rating" : product_rating,
        "price" : product_price,
        "brand": brand,
        "color": color,
        "ear_placement": placement,
        "form_factor": form_factor,
        "impedance": impedance,
        "image_url": image_url,
        "reviews": reviews
    }

    # Append to review data list
    review_data.append(dict_review)


In [16]:
# Menghapus data tanpa product_name
review_data = [item for item in review_data if item["product_name"] != "N/A"]
review_data

[{'product_name': 'Soundcore P30i by Anker Noise Cancelling Earbuds, Strong and Smart Noise Cancelling, Powerful Bass, 45H Playtime, 2-in-1 Case and Phone Stand, IP54, Wireless Earbuds, Bluetooth 5.4 (Black)',
  'rating': '4.4 out of 5 stars',
  'price': '$29.99',
  'brand': 'Soundcore',
  'color': 'Black',
  'ear_placement': 'In Ear',
  'form_factor': 'In Ear',
  'impedance': '16 Ohms',
  'image_url': 'https://m.media-amazon.com/images/I/51HbL1WS+KL._AC_SY300_SX300_QL70_FMwebp_.jpg',
  'reviews': [{'review': 'I had a pair of the AirPod Pros for a couple years when I blew out the left one when my sound was accidentally turned to max one day.  They were out of the Apple Care warranty period and I was waiting until they went on sale over the holidays to replace when I saw the positive reviews on the Soundcore P30i.  I thought for less than the price of just the Apple Care warranty alone that I’d give them a try so I picked up a pair in black.  I was shocked with the quality of the earbud

# iv. File Saving

In [ ]:
# Saving to JSON File
with open("soundcore.json", "w", encoding="utf-8") as review_json:
    json.dump(review_data, review_json, indent=2, ensure_ascii=True)